In [2]:
import pandas as pd
import numpy as np

In [10]:
# Расстояние между точками
def point_point(x1, y1, x2, y2):
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# Периметр многоугольника
def perim(x, y):
    return np.sum(np.sqrt((x - np.roll(x, 1))**2 + (y - np.roll(y, 1))**2))

# Расстояние между точкой и прямой
def point_line(x39, y39, x42, y42, x30, y30):
    p1 = np.array([x39,y39])
    p2 = np.array([x42,y42])
    p3 = np.array([x30,y30])
    return np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)

# Площадь многоугольника
def poly_area(x, y):
    area = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
    return area

# Пересчет расстояния в системе координат под новым углом (относительно прямой между двумя точками) 
def new_coord(x39, y39, x42, y42):
    rads = np.arctan(np.linalg.solve(np.array([[1, x39], [1, x42]]), np.array([y39, y42]))[1])
    return np.cos(rads)

In [11]:
folders = ['Ilya', 'Ilya2', 'Ilya3', 'Ilya4', 'Barbara', 'Barbara2', 'Barbara3', 'Dmitry', 'Liza', 'Masha', 'Ilyas'\
          , 'Ilyas2', 'Ilyas3', 'Ilyas4', 'Ilyas5', 'Ilyas6', 'Ilyas7', 'Ilyas8', 'Ilyas9', 'Ilyas10']
files = ['a_0', 'a_30(1)', 'a_30(2)', 'b_0', 'b_30(1)', 'b_30(2)']

In [13]:
result = pd.DataFrame()

for folder in folders:
    folder_result = pd.DataFrame()
    for file in files:
        df = pd.read_csv(filepath_or_buffer = 'C:/Users/lokri/PycharmProjects/Pract/'+folder+'/'+file+'.csv', sep= ',', index_col= 0)

        # Вычисление нормировочных множителей (для вертикали и горизонтали)
        norm_vert = df.iloc[:, 0:6].apply(lambda x: point_line(x.x39, x.y39, x.x42, x.y42, x.x30, x.y30), axis = 1)
        norm_hor = point_point(df.x39, df.y39, df.x42, df.y42)
        
        coordinate_correction = df.iloc[:, 0:4].apply(lambda x: new_coord(x.x39, x.y39, x.x42, x.y42), axis = 1)

        # Формирование нового датафрейма значениями нормированных расстояний (их проекций на две оси) между каждой из точек контура губ
        columns = [j[1:] for j in list(df.columns[6:-2:2])]
        df_new = pd.DataFrame()
        for i in range(len(columns)):
            for j in range(i+1, len(columns)):
                df_new['x' + columns[i] + '_' + columns[j]] = np.abs(df['x' + columns[i]] - df['x' + columns[j]])*coordinate_correction/norm_hor
                df_new['y' + columns[i] + '_' + columns[j]] = np.abs(df['y' + columns[i]] - df['y' + columns[j]])*coordinate_correction/norm_vert

        # Вычисление нормированных значений периметра контура губ
        df_new['perimeter'] = df.iloc[:, 6:-2].apply(lambda x: perim(np.array(x[0::2]), np.array(x[1::2])), axis = 1)/norm_hor

        # Вычисление нормированных значений площади, ограниченной контуром губ
        df_new['square'] = df.iloc[:, 6:-2].apply(lambda x: poly_area(np.array(x[0::2]), np.array(x[1::2])), axis = 1)/(norm_vert*norm_hor)

        df_new['target'] = df.target

        dummy_df = df_new.groupby('target').agg(['mean', 'std']).reset_index()
        dummy_df.columns = ['_'.join(col) for col in dummy_df.columns.values]
        dummy_df.insert(0, 'angle_type', file)
        folder_result = pd.concat([folder_result,dummy_df])
        
    folder_result.to_csv(path_or_buf='C:/Users/lokri/PycharmProjects/Pract/'+folder+'/'+folder+'_full.csv', index = False)
    folder_result.insert(0, 'person', folder)
    result = pd.concat([result,folder_result])
    
result['target_'] = pd.Categorical(result['target_'], ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'not_num'])
result = result.sort_values(by = ['person', 'angle_type', 'target_'])

result.to_csv(path_or_buf='C:/Users/lokri/PycharmProjects/Pract/full_DB.csv', index = False)